In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [19]:
data_suja_completa=pd.read_csv("speeddating.csv")
# data_suja_completa.head()

/Users/beatrizrianhobernadinho/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (3,10,11,15,16,17,18,19,39,40,41,42,43,44,51,52,53,54,55,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,107,109) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
data_suja_limpo=data_suja_completa[data_suja_completa!= "?" ]
data_suja_limpo.dropna(inplace=True)

/Users/beatrizrianhobernadinho/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [21]:
# data_suja_limpo.columns

In [22]:
data=data_suja_limpo.drop(['d_expected_num_matches','d_expected_num_interested_in_me',
                           'd_expected_happy_with_sd_people',"expected_num_matches",
                           "expected_num_interested_in_me", "guess_prob_liked","decision", 
                           "decision_o","expected_happy_with_sd_people", "samerace","wave",
                           'has_null','d_d_age', 'd_guess_prob_liked'], axis=1)


In [23]:
def dummify(data, column_name):
    """
        Converte a coluna column_name em dummies / one-hot e as adiciona ao dataframe
        retorna uma copia do  df original *sem* a coluna que foi dummified
    """
    df = data.copy()
    # Nota: se você for um econometrista, pode se interessar por usar a opção dropFirst = True na get_dummies
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

In [24]:
X_num = data[['age', "age_o", 'importance_same_race',
 'importance_same_religion',  'pref_o_attractive', 'pref_o_sincere',
 'pref_o_intelligence', 'pref_o_funny', 'pref_o_ambitious',
 'pref_o_shared_interests',  'attractive_o' ,'sinsere_o' ,'intelligence_o',
 'funny_o' ,'ambitous_o', 'shared_interests_o' ,'attractive_important', 'sincere_important',
 'intellicence_important' ,'funny_important',
 'ambtition_important' ,'shared_interests_important' ,'attractive', 'sincere',
 'intelligence' ,'funny' ,'ambition' ,'attractive_partner', 'sincere_partner',
 'intelligence_partner', 'funny_partner' ,'ambition_partner',
 'shared_interests_partner', 'sports', 'tvsports' ,'exercise', 'dining',
 'museums', 'art' ,'hiking' ,'gaming' ,'clubbing', 'reading', 'tv' ,'theater',
 'movies' ,'concerts', 'music' ,'shopping' ,'yoga', 
 'like']]

X_cat = data[['gender', 'race', "race_o", 'field', 'met']]

diferenças = ['d_pref_o_funny' ,'d_pref_o_ambitious','d_pref_o_shared_interests',
'd_pref_o_sincere', 'd_pref_o_intelligence',
'd_pref_o_attractive',"d_age",'d_importance_same_race',
'd_importance_same_religion', 'd_attractive_o',
'd_sinsere_o' ,'d_intelligence_o' ,'d_funny_o', 'd_ambitous_o',
'd_shared_interests_o', 'd_attractive_important' ,'d_sincere_important' ,'d_intellicence_important',
'd_funny_important', 'd_ambtition_important',
'd_shared_interests_important', 'd_attractive', 'd_sincere',
'd_intelligence', 'd_funny' ,'d_ambition', 'd_attractive_partner' ,'d_sincere_partner',
'd_intelligence_partner', 'd_funny_partner', 'd_ambition_partner',
'd_shared_interests_partner', 'd_sports', 'd_tvsports',
'd_exercise' ,'d_dining' ,'d_museums', 'd_art' ,'d_hiking', 'd_gaming',
'd_clubbing' ,'d_reading', 'd_tv' ,'d_theater' ,'d_movies' ,'d_concerts',
'd_music', 'd_shopping', 'd_yoga', 'd_interests_correlate' ,
'd_like']               

lembrar de explicar que tentamos fazer sem as colunas de diferenças.

In [25]:
novo = data
for i in X_cat:
    novo = dummify(novo, i)

In [27]:
for i in diferenças:
    novo = dummify(novo, i)

In [28]:
novo.head()

,age,age_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,...,d_shopping_[9-10],d_yoga_[0-5],d_yoga_[6-8],d_yoga_[9-10],d_interests_correlate_[-1-0],d_interests_correlate_[0-0.33],d_interests_correlate_[0.33-1],d_like_[0-5],d_like_[6-8],d_like_[9-10]
0,21,27,2,4,35,20,20,20,0,5,...,0,1,0,0,0,1,0,0,1,0
1,21,22,2,4,60,0,0,40,0,0,...,0,1,0,0,0,0,1,0,1,0
3,21,23,2,4,30,5,15,40,5,5,...,0,1,0,0,0,0,1,0,1,0
4,21,24,2,4,30,10,20,10,10,20,...,0,1,0,0,0,1,0,0,1,0
5,21,25,2,4,50,0,30,10,0,10,...,0,1,0,0,0,1,0,0,1,0


In [70]:
X = novo.drop('match', axis=1)
y = novo['match']

In [74]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.8238095238095238


In [67]:
y.value_counts(True)

0    0.822519
1    0.177481
Name: match, dtype: float64

In [68]:
sorted(list(zip(model.feature_importances_, X.columns)), reverse=True)

[(0.034235101458067144, 'like'),
 (0.03020210046061515, 'attractive_o'),
 (0.029807585235059046, 'attractive_partner'),
 (0.027955315046807264, 'shared_interests_o'),
 (0.022002251506512076, 'shared_interests_partner'),
 (0.020792523503887266, 'funny_o'),
 (0.019580865090583162, 'funny_partner'),
 (0.01853118252590656, 'interests_correlate'),
 (0.013358905969075427, 'age_o'),
 (0.01308647677620426, 'd_attractive_o_[0-5]'),
 (0.01271623015121315, 'd_shared_interests_o_[0-5]'),
 (0.012693933847004967, 'pref_o_shared_interests'),
 (0.011563944135015047, 'ambitous_o'),
 (0.011486035963303217, 'pref_o_sincere'),
 (0.011222385424748864, 'sinsere_o'),
 (0.010947144160574006, 'exercise'),
 (0.01081412935606895, 'intelligence_o'),
 (0.010740671654716236, 'd_shared_interests_o_[9-10]'),
 (0.010620649652808446, 'pref_o_funny'),
 (0.01021916683372639, 'pref_o_attractive'),
 (0.009978908202439817, 'ambition_partner'),
 (0.009691028460925365, 'pref_o_intelligence'),
 (0.00959631855286697, 'pref_o_am